In [506]:
# importing all packages that may be required
import os
#import gc
import re
import csv
import warnings
import html
import copy
from tqdm import tqdm_notebook
import time
import mailbox
from nltk.corpus import stopwords 
import unicodedata
import pandas as pd
#import multiprocessing
from bs4 import BeautifulSoup
from dask import dataframe as dd
from dask.multiprocessing import get
import nltk
import chardet
# from dask.diagnostics import ProgressBar
from nltk.stem import WordNetLemmatizer as wnl
from nltk import pos_tag as pt
from nltk.corpus import wordnet as wn

In [366]:
cwd = os.getcwd()

In [367]:
# importing mailbox
mail_container = mailbox.mbox(cwd+'/raw_data/podesta-emails.mbox-2016-11-06')

In [368]:
# making working copy
mail_cp = copy.copy(mail_container)

In [460]:
def clean(msg_content):
    if msg_content!=None:
        msg_clean = "".join([ch for ch in str(msg_content) if ord(ch)<= 128])
        for r in ['\\n', '\n', '\r', '\t', '\\t', 'None', 'none']: msg_clean = msg_clean.replace(r, ' ')
        #msg_clean = re.sub("[/\\\:?\"]", '', re.sub(r' +', ' ', msg_clean))
    return msg_clean

In [503]:
#writes to csv
def extract_csv(mail_cp):
    try:
        with open('podesta_mailbox.csv', 'w', encoding='utf-8', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['date', 'from', 'to', 'cc', 'subject', 'originator_org', 'spam_header', 'msg_body'])
            for mail in mail_cp.itervalues():
                if mail.is_multipart():
                    for part in mail.walk():
                        if part.get_content_type()=='text/plain':
                            try:
                                msg_content = part.get_payload(decode=True).decode(part.get_content_charset())
                            except:
                                msg_content = part.get_payload(decode=True)
                else:
                    if mail.get_content_type()=='text/plain':
                        try:
                            msg_content = mail.get_payload(decode=True).decode(mail.get_content_charset())
                        except:
                            msg_content = mail.get_payload(decode=True)
                writer.writerow([mail['Date'], mail['From'], mail['To'], mail['Cc'],
                                 mail['Subject'], mail['X-OriginatorOrg'],
                                 mail['SpamDiagnosticMetadata'],clean(msg_content.lower())]) 
    except:
        pass

In [504]:
# writing to dataframe instead of csv because writing to csv causes line breaks and skipiing cells because of mail content
def extract(mail_cp):
    df = pd.DataFrame(columns=['date', 'from', 'to', 'cc', 'subject', 'originator_org', 'spam_header', 'msg_body'])
    for mail in mail_cp.itervalues():
        if mail.is_multipart():
            for part in mail.walk():
                if part.get_content_type()=='text/plain':
                    try:
                        msg_content = part.get_payload(decode=True).decode(part.get_content_charset())
                    except:
                        msg_content = part.get_payload(decode=True)
        else:
            if mail.get_content_type()=='text/plain':
                try:
                    msg_content = mail.get_payload(decode=True).decode(mail.get_content_charset())
                except:
                    msg_content = mail.get_payload(decode=True)
        d = pd.DataFrame({'date': mail['Date'], 'from': mail['From'],
                          'to': mail['To'], 'cc': mail['Cc'],'subject': mail['Subject'],
                          'originator_org': mail['X-OriginatorOrg'],'spam_header': mail['SpamDiagnosticMetadata'],
                          'msg_body': clean(msg_content.lower())}, index=[0])
        df = pd.concat([df, d])
    return df


In [505]:
# running function and display time
%time df = extract(mail_cp)
# 15 mins - 9.18 gb

/Users/Avinash/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:21: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



CPU times: user 20min 25s, sys: 32.7 s, total: 20min 57s
Wall time: 21min 16s


In [507]:
df.head()

,cc,date,from,msg_body,originator_org,spam_header,subject,to
0,None,"Fri, 3 Apr 2015 14:06:30 +0000",Tina Flournoy <Tina@presidentclinton.com>,sc was the first thing he mentioned. told him to talk to you about greece,None,None,just talked to him,John Podesta <john.podesta@gmail.com>
0,None,"Mon, 22 Feb 2016 23:00:30 +0000",Ira Shapiro <ira@shapiroglobal.com>,"john, sorry we didn't get to visit this morning. i teach once a semester for madeleine, and this...",shapiroglobal.com,NSPM,today's fundraiser,John Podesta <john.podesta@gmail.com>
0,None,"Tue, 17 Jun 2014 01:32:54 +0000","""Sandler, Herbert"" <hms@sandlerfoundation.org>","sent from my ipad begin forwarded message: from: ""knaebel, sergio"" <sknaebel@sandlerfoundation...",None,None,Fwd: NYTimes: The True Cost of Hidden Money,"Heather Boushey <hboushey@equitablegrowth.org>,\n John Podesta <john.podesta@gmail.com>"
0,None,30 Oct 2015 16:50:11 -0400,Delta Air Lines <acs.survey@delta.com>,"sent from my ipad begin forwarded message: from: ""knaebel, sergio"" <sknaebel@sandlerfoundation...",None,None,Tell Us How We Did On Your Flight From ATL to IAD,john.podesta@gmail.com
0,None,"Sat, 28 Jul 2012 09:00:19 -0500 (CDT)","""Gene Karpinski, League of Conservation Voters"" <feedback@lcv.org>","dear john, today marks 100 days until election day and i am pleased to update you on some of our ...",None,None,100 days out...,john.podesta@gmail.com


mail = mail_cp[8]
if mail.is_multipart():
    for part in mail.walk():
        if part.get_content_type()=='text/plain':
            try:
                msg_content = part.get_payload(decode=True).decode(part.get_content_charset())
            except:
                msg_content = part.get_payload(decode=True)
else:
    if mail.get_content_type()=='text/plain':
        try:
            msg_content = mail.get_payload(decode=True).decode(mail.get_content_charset())
        except:
            msg_content = mail.get_payload(decode=True)

clean(msg_content)

msg_content.decode(chardet.detect(msg_content)['encoding'])

In [508]:
# spam headers = https://docs.microsoft.com/en-us/office365/securitycompliance/anti-spam-message-headers

In [509]:
# listing datatypes of dataframe and including in importing csv with 50Mb blocksize
#dtypes={'from': 'object', 'to': 'object', 'subject': 'object', 'originator_org': 'object', 'spam_header': 'object','msg_body': 'object'}
#podesta = pd.read_csv(cwd+'/podesta_mailbox.csv', dtype=dtypes, parse_dates=True)
podesta = df

In [510]:
podesta.head(2)

,cc,date,from,msg_body,originator_org,spam_header,subject,to
0,None,"Fri, 3 Apr 2015 14:06:30 +0000",Tina Flournoy <Tina@presidentclinton.com>,sc was the first thing he mentioned. told him to talk to you about greece,None,None,just talked to him,John Podesta <john.podesta@gmail.com>
0,None,"Mon, 22 Feb 2016 23:00:30 +0000",Ira Shapiro <ira@shapiroglobal.com>,"john, sorry we didn't get to visit this morning. i teach once a semester for madeleine, and this...",shapiroglobal.com,NSPM,today's fundraiser,John Podesta <john.podesta@gmail.com>


In [511]:
def mail_ext(x):
    x = re.search(r'\S+@\S+', str(x).lower())
    if x:
        return (x.group().replace('>','').replace('<','').lower())

def name_r(em):
    em = str(em).lower()
    t = re.sub(r'\S+@\S+', '', em)
    if  t==None or t==" " or t=="":
        return re.search(r'.*?(?=@)', em).group().replace('>','').replace('<','').replace('.',' ')
    else:
        return re.sub(r'\S+@\S+', '', em)    

In [512]:
podesta['to_email'] = podesta.to.apply(lambda x: mail_ext(x))
podesta['from_email'] = podesta['from'].apply(lambda x: mail_ext(x))

podesta['to_name'] = podesta.to.apply(lambda x: name_r(x))
podesta['from_name'] = podesta['from'].apply(lambda x: name_r(x))


In [513]:
podesta.head(2)

,cc,date,from,msg_body,originator_org,spam_header,subject,to,to_email,from_email,to_name,from_name
0,None,"Fri, 3 Apr 2015 14:06:30 +0000",Tina Flournoy <Tina@presidentclinton.com>,sc was the first thing he mentioned. told him to talk to you about greece,None,None,just talked to him,John Podesta <john.podesta@gmail.com>,john.podesta@gmail.com,tina@presidentclinton.com,john podesta,tina flournoy
0,None,"Mon, 22 Feb 2016 23:00:30 +0000",Ira Shapiro <ira@shapiroglobal.com>,"john, sorry we didn't get to visit this morning. i teach once a semester for madeleine, and this...",shapiroglobal.com,NSPM,today's fundraiser,John Podesta <john.podesta@gmail.com>,john.podesta@gmail.com,ira@shapiroglobal.com,john podesta,ira shapiro


In [514]:
def get_wordnet_pos(word):
    if word.startswith('J'):
        return wn.ADJ
    elif word.startswith('V'):
        return wn.VERB
    elif word.startswith('N'):
        return wn.NOUN
    elif word.startswith('R'):
        return wn.ADV
    else:
        return wn.NOUN

def stop_rem(msg_content):
    l=[]
    stop = nltk.word_tokenize(msg_content)
    stop_words = set(stopwords.words('english'))
    filtered = [w for w in stop if w not in stop_words]
    ptag = pt(filtered)
    for word in ptag:
        lem = wnl().lemmatize(word[0], get_wordnet_pos(word[1]))
        if lem!=None or lem!='' or lem!=' ':
            l.append(str(lem.lower()))
        else:
            pass

    filsent = ' '.join(w for w in l)
    return filsent

In [515]:
podesta['stprem_body']=podesta.msg_body.progress_apply(lambda x: stop_rem(x))

In [516]:
pd.set_option("max_colwidth",102)
podesta.head()[['msg_body','stprem_body']]

,msg_body,stprem_body
0,sc was the first thing he mentioned. told him to talk to you about greece,sc first thing mention . told talk greece
0,"john, sorry we didn't get to visit this morning. i teach once a semester for madeleine, and this...","john , sorry n't get visit morning . teach semester madeleine , morning , exit early . great katie..."
0,"sent from my ipad begin forwarded message: from: ""knaebel, sergio"" <sknaebel@sandlerfoundation...","sent ipad begin forward message : : `` knaebel , sergio '' < sknaebel @ sandlerfoundation.org < ma..."
0,"sent from my ipad begin forwarded message: from: ""knaebel, sergio"" <sknaebel@sandlerfoundation...","sent ipad begin forward message : : `` knaebel , sergio '' < sknaebel @ sandlerfoundation.org < ma..."
0,"dear john, today marks 100 days until election day and i am pleased to update you on some of our ...","dear john , today mark 100 day election day please update recent efforts.thank continue partner cr..."


In [517]:
#podesta_dd = dd.from_pandas(podesta, npartitions=4)

In [518]:
#podesta_dd

In [519]:
#%time pod_lemat = podesta_dd.apply(lambda df: lemat(df.msg_body), meta=pd.Series(dtype='object', name='lemat_body'), axis=1)

#pod_lemat.head(2).compute()

#with ProgressBar():
#    podesta['lemat_body'] = pod_lemat.compute(scheduler='processes')

In [520]:
# podesta['lenat_body']=podesta_dd.map_partitions(lambda df: df.msg_body.apply(lemat), meta=pd.Series(dtype='object', name='lemat_body'))


In [521]:
% who

BeautifulSoup	 ProgressBar	 chardet	 clean	 copy	 csv	 cwd	 dd	 df	 
dtypes	 e	 em	 extract	 extract_c	 extract_csv	 get	 get_wordnet_pos	 html	 
lemat	 mail	 mail_container	 mail_cp	 mail_ext	 mailbox	 msg_content	 name_r	 nltk	 
os	 part	 pd	 pod_lemat	 podesta	 podesta_dd	 pt	 re	 stop_rem	 
stopwords	 time	 tqdm	 tqdm_notebook	 unicodedata	 warnings	 wn	 wnl	 


In [525]:
!git commit -m 'replaced writing to csv with dataframe, to reduce errors'
!git push -f origin master

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
	deleted:    mbox_extraction.ipynb
	modified:   mbox_extraction_pandas.ipynb

Untracked files:
	.ipynb_checkpoints/
	email_extract_211218.ipynb
	podesta_mailbox.csv
	raw_data/

no changes added to commit
Everything up-to-date


In [526]:
!git push origin master -f

Everything up-to-date
